# Stock Market Question Answering Agent

- Answer questions about individual stocks using (mostly) OpenBB functions
- uses the OpenAI tools functionality 
- define a set of functions
- when prompting, give OpenAI the set of functions and their descriptions in the prompt and also in the structured JSON format
- if OpenAI needs to run a function to respond to the prompt, it will provide the function signature and ask you to provide the return values before continuing

  

In [2]:
import os
import dotenv
import warnings
from datetime import datetime, timedelta

import pandas as pd

import IPython
from IPython.display import HTML, Image, Markdown, display

import requests
import json

import openbb
from openbb import obb
from openbb_core.app.model.obbject import OBBject

import openai
from openai import OpenAI

import langchain
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# paid API for edgar filings
import sec_api
from sec_api import QueryApi, ExtractorApi

# free API for edgar filing
from sec_downloader import Downloader
import sec_parser as sp

dotenv.load_dotenv()

# class to make a tool and utility functions
from BB_agent_tool import BB_agent_tool, get_response, agent_query

import pdb

# turn off excessive warnings
warnings.filterwarnings('ignore')


In [3]:
print(f'pandas         {pd.__version__}')
print(f'obb            {obb.system.version}')
print(f'openai         {openai.__version__}')
print(f'langchain      {langchain.__version__}')


pandas         2.2.2
obb            4.2.1
openai         1.30.3
langchain      0.2.1


# Connect to OpenBB

In [4]:
obb


OpenBB Platform v4.2.1

Utilities:
    /account
    /user
    /system
    /coverage

Routers:
    /crypto
    /currency
    /derivatives
    /economy
    /equity
    /etf
    /fixedincome
    /index
    /news
    /regulators

Extensions:
    - commodity@1.1.1
    - crypto@1.2.1
    - currency@1.2.1
    - derivatives@1.2.1
    - economy@1.2.1
    - equity@1.2.1
    - etf@1.2.1
    - fixedincome@1.2.1
    - index@1.2.1
    - news@1.2.1
    - regulators@1.2.1

    - benzinga@1.2.1
    - econdb@1.1.1
    - federal_reserve@1.2.1
    - fmp@1.2.1
    - fred@1.2.1
    - intrinio@1.2.1
    - oecd@1.2.1
    - polygon@1.2.1
    - sec@1.2.1
    - tiingo@1.2.1
    - tradingeconomics@1.2.1
    - yfinance@1.2.1    

In [5]:
# login with email and password
obb.account.login(email=os.environ['OPENBB_USER'], password=os.environ['OPENBB_PW'], remember_me=True)


In [6]:
obb.user


UserSettings

id: 06653664-f525-755c-8000-f24c3dd11849
profile: {'hub_session': {'username': 'drucev', 'email': 'drucev@hotmail.com', 'primary_usage': 'personal', 'user_uuid': 'c866b4d2-c09b-4b13-abb7-a93f1ac3c2b7', 'token_type': 'bearer', 'access_token': SecretStr('**********')}}
credentials: {'econdb_api_key': None, 'intrinio_api_key': None, 'fred_api_key': SecretStr('**********'), 'fmp_api_key': SecretStr('**********'), 'benzinga_api_key': None, 'polygon_api_key': SecretStr('**********'), 'tradingeconomics_api_key': None, 'tiingo_token': SecretStr('**********')}
preferences: {'cache_directory': '/Users/drucev/OpenBBUserData/cache', 'chart_style': 'dark', 'data_directory': '/Users/drucev/OpenBBUserData', 'export_directory': '/Users/drucev/OpenBBUserData/exports', 'metadata': True, 'output_type': 'OBBject', 'plot_enable_pywry': True, 'plot_open_export': False, 'plot_pywry_height': 762, 'plot_pywry_width': 1400, 'request_timeout': 60, 'show_warnings': False, 'table_style': 'dark', 'use

In [7]:
obb.account


/account
    login
    logout
    save
    refresh
    

In [8]:
# Change a credential - only need once, gets stored in openbb cloud
# obb.user.credentials.polygon_api_key = os.environ['POLYGON_API_KEY']
# obb.user.credentials.alpha_vantage_api_key = os.environ['ALPHAVANTAGE_API_KEY']
# obb.user.credentials.fred_api_key = os.environ['FRED_API_KEY']
# obb.user.credentials.tiingo_token = os.environ['TIINGO_API_KEY']
# obb.user.credentials.fmp_api_key = os.environ['FMP_API_KEY']
# obb.user.credentials.biztoc_api_key = os.environ['BIZTOC_API_KEY']

# Save account changes to the Hub
# obb.account.save()

# Refresh account with latest changes since login
# obb.account.refresh()

# Logout
# obb.account.logout()


In [9]:
obb.equity.search("Merck", provider="sec").to_df().head(3)


,symbol,name,cik
0,MRK,"Merck & Co., Inc.",310158


In [10]:
symbol = "MRK"
company = "Merck"


In [11]:
obj = obb.equity.price.quote(symbol)


In [12]:
obj


OBBject

id: 06653667-e67f-7e9f-8000-a683a5fe964d
results: [{'symbol': 'MRK', 'asset_type': None, 'name': 'Merck & Co., Inc.', 'excha...
provider: fmp
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'fmp'}, 'standa...

In [13]:
obj.results


[FMPEquityQuoteData(symbol=MRK, asset_type=None, name=Merck & Co., Inc., exchange=NYSE, bid=None, bid_size=None, bid_exchange=None, ask=None, ask_size=None, ask_exchange=None, quote_conditions=None, quote_indicators=None, sales_conditions=None, sequence_number=None, market_center=None, participant_timestamp=None, trf_timestamp=None, sip_timestamp=None, last_price=129.49, last_tick=None, last_size=None, last_timestamp=2024-05-24 20:00:02+00:00, open=131.45, high=131.45, low=129.16, close=None, volume=5829244, exchange_volume=None, prev_close=131.09, change=-1.6, change_percent=-0.012204999999999999, year_high=133.1, year_low=99.14, price_avg50=127.7616, price_avg200=115.32135, avg_volume=8199988, market_cap=327973566900.0, shares_outstanding=2532810000, eps=0.9, pe=143.88, earnings_announcement=2024-07-30 12:30:00+00:00)]

In [14]:
obj.json()

'{"id":"06653667-e67f-7e9f-8000-a683a5fe964d","results":[{"symbol":"MRK","asset_type":null,"name":"Merck & Co., Inc.","exchange":"NYSE","bid":null,"bid_size":null,"bid_exchange":null,"ask":null,"ask_size":null,"ask_exchange":null,"quote_conditions":null,"quote_indicators":null,"sales_conditions":null,"sequence_number":null,"market_center":null,"participant_timestamp":null,"trf_timestamp":null,"sip_timestamp":null,"last_price":129.49,"last_tick":null,"last_size":null,"last_timestamp":"2024-05-24T20:00:02Z","open":131.45,"high":131.45,"low":129.16,"close":null,"volume":5829244,"exchange_volume":null,"prev_close":131.09,"change":-1.6,"change_percent":-0.012204999999999999,"year_high":133.1,"year_low":99.14,"price_avg50":127.7616,"price_avg200":115.32135,"avg_volume":8199988,"market_cap":327973566900.0,"shares_outstanding":2532810000,"eps":0.9,"pe":143.88,"earnings_announcement":"2024-07-30T12:30:00Z"}],"provider":"fmp","warnings":null,"chart":null,"extra":{"metadata":{"arguments":{"provid

In [15]:
# use REST API on server running locally
# uvicorn openbb_core.api.rest_api:app --host 0.0.0.0 --port 8000 --reload
# REST API documentation - http://127.0.0.1:8000/docs
# openapi.json : http://127.0.0.1:8000/openapi.json

# not turning on authentication
# msg = "some_user:some_pass"
# msg_bytes = msg.encode('ascii')
# base64_bytes = base64.b64encode(msg_bytes)
# base64_msg = base64_bytes.decode('ascii')

url = f"http://127.0.0.1:8000/api/v1/equity/price/quote?provider=yfinance&symbol={symbol}"
# headers = {"accept": "application/json", "Authorization": f"Basic {base64_msg}"}
headers = {"accept": "application/json"}

response = requests.get(url=url, headers=headers)

response.json()


{'results': [{'symbol': 'MRK',
   'asset_type': 'EQUITY',
   'name': 'Merck & Co., Inc.',
   'exchange': 'NYQ',
   'bid': 129.35,
   'bid_size': 800,
   'ask': 129.49,
   'ask_size': 1800,
   'last_price': 129.49,
   'open': 131.45,
   'high': 131.45,
   'low': 129.16,
   'volume': 5946890,
   'prev_close': 131.09,
   'year_high': 133.1,
   'year_low': 99.14,
   'ma_50d': 127.9412,
   'ma_200d': 115.4383,
   'volume_average': 8213074.0,
   'volume_average_10d': 6263710.0,
   'currency': 'USD'}],
 'provider': 'yfinance',
 'warnings': None,
 'chart': None,
 'extra': {'metadata': {'arguments': {'provider_choices': {'provider': 'yfinance'},
    'standard_params': {'symbol': 'MRK'},
    'extra_params': {'source': 'iex'}},
   'duration': 279763250,
   'route': '/equity/price/quote',
   'timestamp': '2024-05-26T12:42:41.364714'}}}

In [16]:
data = obb.equity.price.historical(symbol, provider="polygon")
data.to_dataframe()


,open,high,low,close,volume,vwap,transactions
date,,,,,,,
2023-05-26,112.0000,113.19,110.6501,111.07,6378339.0,111.3186,69766
2023-05-30,110.2222,110.34,109.0800,110.09,166016.0,109.8427,3155
2023-05-31,108.4700,111.15,108.4700,110.41,14563536.0,110.3172,91481
2023-06-01,110.6600,111.59,109.8800,110.93,6960063.0,110.9360,76972
2023-06-02,111.1100,112.73,110.7200,112.52,5792792.0,112.1935,74396
...,...,...,...,...,...,...,...
2024-05-20,130.8100,131.74,130.6500,130.99,4333726.0,131.2016,58819
2024-05-21,131.6800,132.79,130.4800,130.76,5280473.0,131.1038,66107
2024-05-22,130.1700,131.60,130.1500,131.10,4329193.0,131.1204,57070


In [17]:
# use the local rest server
data = []
symbol2="SPY"
url = f"http://127.0.0.1:8000/api/v1/equity/price/historical?provider=polygon&symbol={symbol2}"
headers = {"accept": "application/json"}

response = requests.get(url, headers=headers, timeout=3)

if response.status_code == 200:
  data = OBBject.model_validate(response.json())

data.to_df()


,close,high,low,open,transactions,volume,vwap
date,,,,,,,
2023-05-26,420.02,420.7700,415.250,415.33,627251,93829975.0,418.9092
2023-05-30,420.18,422.5800,418.735,422.03,609041,72207960.0,420.4468
2023-05-31,417.85,419.2200,416.220,418.28,665147,110782530.0,418.0147
2023-06-01,421.82,422.9200,416.790,418.09,653406,88758018.0,420.4660
2023-06-02,427.92,428.7400,423.950,424.50,678309,91426171.0,426.8396
...,...,...,...,...,...,...,...
2024-05-20,530.06,531.5601,529.170,529.57,331748,37711090.0,530.2811
2024-05-21,531.36,531.5200,529.070,529.28,306611,33382000.0,530.4508
2024-05-22,529.83,531.3800,527.600,530.65,340278,47734744.0,529.7849


In [18]:
results = obb.equity.search(query='marvell', is_symbol=False, provider='sec', use_cache=True)
[(r.symbol, r.name) for r in results.results]


[('MRVL', 'Marvell Technology, Inc.')]

In [19]:
# multiple symbols
quotes = obb.equity.price.quote("td,schw,jpm,ms", provider="fmp")
quotes.to_df()


,symbol,name,exchange,last_price,last_timestamp,open,high,low,volume,prev_close,...,year_high,year_low,price_avg50,price_avg200,avg_volume,market_cap,shares_outstanding,eps,pe,earnings_announcement
0,TD,The Toronto-Dominion Bank,NYSE,56.56,2024-05-24 20:00:02+00:00,55.46,56.82,55.43,2344166,55.01,...,66.15,54.12,58.0400,59.83815,2951465,1.002300e+11,1772100000,4.36,12.97,2024-05-23 00:00:00+00:00
1,SCHW,The Charles Schwab Corporation,NYSE,72.27,2024-05-24 20:00:01+00:00,72.64,73.09,72.17,6123549,72.34,...,79.49,48.66,73.3880,63.64275,7378571,1.284440e+11,1777280000,2.39,30.24,2024-07-16 12:30:00+00:00
2,JPM,JPMorgan Chase & Co.,NYSE,200.71,2024-05-24 20:00:02+00:00,197.75,200.76,197.56,7218799,196.92,...,205.88,134.40,194.2518,167.96075,9108733,5.763729e+11,2871670000,16.56,12.12,2024-07-12 12:30:00+00:00
3,MS,Morgan Stanley,NYSE,98.97,2024-05-24 20:00:02+00:00,99.45,99.91,98.81,3547499,98.92,...,103.25,69.42,93.4398,86.17185,7814188,1.608421e+11,1625160000,5.50,17.99,2024-07-16 12:30:00+00:00


In [20]:
# multiple providers

df = pd.DataFrame()

df["yfinance"] = (
  obb.equity.fundamental.balance(symbol, provider="yfinance", limit=3)
  .to_df().get("total_assets")
)

df["fmp"] = (
  obb.equity.fundamental.balance(symbol, provider="fmp", limit=3)
  .to_df().get("total_assets")
)

df["polygon"] = (
  obb.equity.fundamental.balance(symbol, provider="polygon", limit=3)
  .to_df().get("total_assets")
)

df

,yfinance,fmp,polygon
0,1.066750e+11,1.066750e+11,1.066750e+11
1,1.091600e+11,1.091600e+11,1.091600e+11
2,1.056940e+11,1.056940e+11,1.056940e+11
3,9.158800e+10,NaN,NaN


In [21]:
obb.news.company(symbol, provider='polygon', limit=5).to_df()


,title,text,images,url,symbols,source,tags,id,amp_url,publisher
date,,,,,,,,,,
2024-05-20 14:45:00+00:00,AstraZeneca (AZN) to Build $1.5B Cancer Drug P...,This new plant will be AstraZeneca's (AZN) fir...,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2276201/astra...,"AZN,PFE,MRK",Zacks Equity Research,None,mRCY0S03pJh-0mZOQdgzBpD1tc09a63S4-xzHV1B9sw,https://www.zacks.com/amp/stock/news/2276201/a...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-22 13:50:11+00:00,Why Merck (MRK) is a Top Momentum Stock for th...,The Zacks Style Scores offers investors a way ...,[{'url': 'https://staticx-tuner.zacks.com/imag...,https://www.zacks.com/stock/news/2277274/why-m...,MRK,Zacks Equity Research,None,u-Pzf5KpgMBaRgI4tI6hnc8dCjVUV-gYNhCrshvOOpg,https://www.zacks.com/amp/stock/news/2277274/w...,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-24 09:50:00+00:00,Billionaire Ken Griffin Is Buying These Divide...,The billionaire hedge fund manager doesn't nee...,[{'url': 'https://g.foolcdn.com/editorial/imag...,https://www.fool.com/investing/2024/05/24/bill...,"BAC,MRK,HES,CVX",newsfeedback@fool.com (Keith Speights),investing,bZDVTjsKEErP9Ogrz6RcUanUPRyov0URt8izBs3fs-0,None,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-24 15:03:44+00:00,Sell Alert: 2 Dividend Stocks To Sell In May A...,September is the consistent worst month for th...,[{'url': 'https://static.seekingalpha.com/cdn/...,https://seekingalpha.com/article/4695496-sell-...,"ABBV,AMGN,BAH,BMY,DKS,IBM,PFE,XOM,WSM,MRK",Robert & Sam Kovacs,None,kZi_Z2OKRAq2j3OfMEDWO6i2i4w8OWSVArwDG6pFtYs,None,{'favicon_url': 'https://s3.polygon.io/public/...
2024-05-24 17:41:19+00:00,Why Is Cancer-Focused Merus Stock Trading Over...,"Thursday, Merus N.V. (NASDAQ:MRUS) announced t...",[{'url': 'https://cdn.benzinga.com/files/image...,https://www.benzinga.com/analyst-ratings/analy...,"MRK,MRUS",Vandana Singh,"Analyst Color,Biotech,Health Care,Analyst Rati...",-YDejnW7jjgd3AiAI2lljjroXf_d6b3CFyZ5L7svJbs,https://www.benzinga.com/amp/content/39012157,{'favicon_url': 'https://s3.polygon.io/public/...


# Prompt OpenAI 

In [22]:
MODEL = "gpt-4o"

# MAX_INPUT_TOKENS = 65536
MAX_OUTPUT_TOKENS = 4096    # max in current model
MAX_RETRIES = 3
TEMPERATURE = 0

client = OpenAI()


In [23]:
messages = [{"role": "user", "content": "what is the airspeed velocity of an unladen swallow?"}]
response = get_response(client, messages, tools=[])
response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\\$")

print(response_str)

The question about the airspeed velocity of an unladen swallow is famously asked in the movie "Monty Python and the Holy Grail." The movie makes a distinction between African and European swallows, and the query is often presented in a humorous context.

However, if you're looking for a more scientific answer: studies on bird flight suggest that the European swallow (Hirundo rustica) has a cruising airspeed velocity of around 11 meters per second, or approximately 24 miles per hour (mph). This figure can vary based on environmental conditions and effort exerted by the bird.

Different species and individual conditions can lead to different speeds, so this is a general estimate rather than an exact number.


# Question Answering Agent

In [24]:
# make a custom tool

def get_10k_item1_from_symbol(symbol):
    """
    Get item 1 of the latest 10-K annual report filing for a given symbol.

    Args:
        symbol (str): The symbol of the equity.

    Returns:
        str: The item 1 of the latest 10-K annual report filing, or None if not found.

    """
    item1_text = None
    try:
        # sec needs you to identify yourself for rate limiting
        dl = Downloader(os.getenv("SEC_FIRM"), os.getenv("SEC_USER"))
        html = dl.get_filing_html(ticker=symbol, form="10-K")
        elements: list = sp.Edgar10QParser().parse(html)
        tree = sp.TreeBuilder().build(elements)
        sections = [n for n in tree.nodes if n.text.startswith("Item")]
        item1_node = sections[0]
        item1_text = "\n".join([n.text for n in sections[0].get_descendants()])
    except:
        return None
    # always return a list of dicts
    return [{'item1': item1_text}]

fn_metadata = {
    "name": "get_10k_item1_from_symbol",
    "description": "Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing.",
    "openapi_path" : None,
    "callable": get_10k_item1_from_symbol,
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The symbol to get the 10-K item 1 for"
            }
        },
    "default_parameters": {},
    "example_parameter_values": [{
        "symbol": "MSFT",
    }],
}

tool = BB_agent_tool(**fn_metadata)
tool_response = tool(symbol="MSFT")
len(json.loads(tool_response)[0]['item1'])


143226

In [25]:
user_message = "Please summarize item 1 from the latest MSFT annual report"
agent_query(client, user_message, verbose=True)



Role: You are an AI stock market assistant tasked with providing investors
with up-to-date, detailed information on individual stocks.

Objective: Assist data-driven stock market investors by giving accurate,
complete, but concise information relevant to their questions about individual
stocks.

Capabilities: You are given a number of tools as functions. Use as many tools
as needed to ensure all information provided is timely, accurate, concise,
relevant, and responsive to the user's query.

Instructions:
1. Input validation. Determine if the input is asking about a specific company
or stock ticker. If not, respond in a friendly, positive, professional tone
that you don't have information to answer and suggest alternative services
or approaches.

2. Symbol extraction. If the query is valid, extract the company name or ticker
symbol from the question. If a company name is given, look up the ticker symbol
using a tool. If the ticker symbol is not found based on the company, try to
corre

### Summary of Item 1 from Microsoft's Latest 10-K Annual Report:

#### Forward-Looking Statements:
- **Definition**: The report contains forward-looking statements regarding business plans and expected results.
- **Caution**: These statements are subject to risks and uncertainties and must not be taken as guarantees.

#### Business Overview:
- **Mission**: Empower every person and organization globally to achieve more.
- **Core Areas**: AI, cloud services, productivity tools, and personal computing.
- **Focus**: Creating platforms and tools, enhancing educational and health outcomes, efficiency in the public sector, and supporting business competitiveness.

#### AI and Cloud Services:
- **Innovation**: Integration of AI across products like Microsoft Teams, Outlook, Bing, and Xbox.
- **AI Principles**: Fairness, reliability, safety, privacy, security, inclusiveness, transparency, and accountability.
- **Cloud Services**: Microsoft Azure offers hybrid consistency, AI capabilities, and trusted security.

#### Product Offerings:
- **Software and Devices**: Operating systems, server applications, business solutions, and devices like PCs, tablets, and gaming consoles.
- **Microsoft 365**: Combines Office 365, Windows, and security services to enhance productivity and collaboration.
- **Gaming**: Focused on expanding gaming experiences and content through investments in studios and technologies like Xbox Game Pass.
- **Search and Advertising**: Services like Bing and Microsoft Edge are key for user engagement and advertising revenue.

#### Societal Commitments:
- **Sustainability**: Goals to be carbon negative, water positive, and zero waste by 2030, along with significant environmental sustainability initiatives.
- **Racial Equity**: Projects to improve racial equity and increase representation and inclusion within the company and communities.
- **Digital Skills**: Initiatives to support job seekers and train workers on digital and AI skills.

#### Human Capital:
- **Workforce**: Approximately 221,000 employees globally, with efforts focused on creating an inclusive, rewarding work environment.
- **Culture**: Emphasis on a growth mindset, diversity, and inclusion, ensuring pay equity, and supporting employee wellbeing and development.

#### Operating Segments:
- **Productivity and Business Processes**: Includes Office, LinkedIn, and Dynamics business solutions.
- **Intelligent Cloud**: Encompasses Azure services, server products, and enterprise services.
- **More Personal Computing**: Covers Windows, devices like Surface and HoloLens, gaming through Xbox, and search and advertising services.

#### Challenges and Risks:
- **Competition**: Facing intense competition in software, services, and devices.
- **Regulatory**: Continued scrutiny and regulatory compliance are key in different geographies.
- **Cybersecurity**: Constant threat from cyberattacks requires significant investment in technology and security measures.

# Add more OpenBB tools
Map OpenBB functions to OpenAI tools

In [26]:
openai_tools=[]
tool_descs=""

for v in BB_agent_tool.agent_registry.values():
        t = v.tooldict
        openai_tools.append(t)
        tname = t['function']['name']
        tdesc = t['function']['description']
        tool_descs += f"{tname} : {tdesc}"
        if v.example_code:
            tool_descs += f" Usage: {v.example_code}"
        tool_descs += "\n---\n"

tool_descs

'get_10k_item1_from_symbol : Given a stock symbol, gets item 1 of the company\'s latest 10-K annual report filing. Usage: get_10k_item1_from_symbol(symbol="MSFT") -> [{"item1": "Note About Forward-Looking Statements\\nThis report includes estimates, projections, statements relating to our business plans, objectives, and expected operating results that are \\u201cforward-looking statements\\u201d within the meaning of the Private Securities Litigation Reform Act of 1995, Section 27A of the Securities Act of 1933, and Section 21E of the Securities Exchange Act of 1934. Forward-looking statements may appear throughout this report, including the following sections: \\u201cBusiness\\u201d (Part I, Item 1 of this Form 10-K), \\u201cRisk Factors\\u201d (Part I, Item 1A of this Form 10-K), and \\u201cManagement\\u2019s Discussion and Analysis of Financial Condition and Results of Operations\\u201d (Part II, Item 7 of this Form 10-K). These forward-looking statements generally are identified by

In [29]:
# load the OpenAPI / swagger spec from
# http://127.0.0.1:8000/openapi.json
with open("openapi.json", 'r') as file:
    data = json.load(file)

str(data)[:2000]


"{'openapi': '3.1.0', 'info': {'title': 'OpenBB Platform API', 'description': 'This is the OpenBB Platform API.', 'termsOfService': 'http://example.com/terms/', 'contact': {'name': 'OpenBB Team', 'url': 'https://openbb.co/', 'email': 'hello@openbb.co'}, 'license': {'name': 'MIT', 'url': 'https://github.com/OpenBB-finance/OpenBBTerminal/blob/develop/LICENSE'}, 'version': '1'}, 'servers': [{'url': 'http://localhost:8000', 'description': 'Local OpenBB development server'}], 'paths': {'/api/v1/commodity/lbma_fixing': {'get': {'tags': ['commodity'], 'summary': 'Lbma Fixing', 'description': 'Daily LBMA Fixing Prices in USD/EUR/GBP.', 'operationId': 'commodity_lbma_fixing', 'parameters': [{'name': 'provider', 'in': 'query', 'required': False, 'schema': {'enum': ['nasdaq'], 'const': 'nasdaq', 'type': 'string', 'default': 'nasdaq', 'title': 'Provider'}}, {'name': 'asset', 'in': 'query', 'required': False, 'schema': {'enum': ['gold', 'silver'], 'type': 'string', 'description': 'The metal to get 

In [30]:
# list all the equity functions
for path_str, fn_json in data['paths'].items():
    if path_str.find('equity') != -1:
        print(path_str)


/api/v1/equity/calendar/ipo
/api/v1/equity/calendar/dividend
/api/v1/equity/calendar/splits
/api/v1/equity/calendar/earnings
/api/v1/equity/compare/peers
/api/v1/equity/compare/groups
/api/v1/equity/estimates/price_target
/api/v1/equity/estimates/historical
/api/v1/equity/estimates/consensus
/api/v1/equity/estimates/analyst_search
/api/v1/equity/estimates/forward_sales
/api/v1/equity/estimates/forward_eps
/api/v1/equity/darkpool/otc
/api/v1/equity/discovery/gainers
/api/v1/equity/discovery/losers
/api/v1/equity/discovery/active
/api/v1/equity/discovery/undervalued_large_caps
/api/v1/equity/discovery/undervalued_growth
/api/v1/equity/discovery/aggressive_small_caps
/api/v1/equity/discovery/growth_tech
/api/v1/equity/discovery/top_retail
/api/v1/equity/discovery/upcoming_release_days
/api/v1/equity/discovery/filings
/api/v1/equity/fundamental/multiples
/api/v1/equity/fundamental/balance
/api/v1/equity/fundamental/balance_growth
/api/v1/equity/fundamental/cash
/api/v1/equity/fundamental/r

In [31]:
data['paths']['/api/v1/equity/search']


{'get': {'tags': ['equity'],
  'summary': 'Search',
  'description': 'Search for stock symbol, CIK, LEI, or company name.',
  'operationId': 'equity_search',
  'parameters': [{'name': 'provider',
    'in': 'query',
    'required': True,
    'schema': {'enum': ['cboe', 'intrinio', 'nasdaq', 'sec', 'tmx', 'tradier'],
     'type': 'string',
     'title': 'Provider'}},
   {'name': 'query',
    'in': 'query',
    'required': False,
    'schema': {'type': 'string',
     'description': 'Search query.',
     'default': '',
     'title': 'Query'},
    'description': 'Search query.'},
   {'name': 'is_symbol',
    'in': 'query',
    'required': False,
    'schema': {'type': 'boolean',
     'description': 'Whether to search by ticker symbol.',
     'default': False,
     'title': 'Is Symbol'},
    'description': 'Whether to search by ticker symbol.'},
   {'name': 'use_cache',
    'in': 'query',
    'required': False,
    'schema': {'anyOf': [{'type': 'boolean'}, {'type': 'null'}],
     'descriptio

In [32]:
fn_metadata = {
    "name": "get_equity_search_symbol",
    "description": "Given a search string, get the stock symbol of the top company whose name best matches the search string.",
    "openapi_path" : '/api/v1/equity/search',
    "parameters": {
        "query": {
            "type": "string",
            "description": "The search string to match to the stock symbol."
            },""
        "limit": {
            "type": "integer",
            "description": "The number of results to return. Pick a small number from 1 to 10 and choose the best response."
            }
        },
    "default_parameters": {
        "provider": "sec"
    },
    "example_parameter_values": [{
        "query": "Broadcom",
    }],
    # "singular": 1,
}

tool = BB_agent_tool(**fn_metadata)



In [33]:
BB_agent_tool.agent_registry['get_equity_search_symbol'](query="Southwest Air")

'[{"symbol": "LUV", "name": "SOUTHWEST AIRLINES CO", "cik": "92380"}]'

In [34]:
agent_query(client, "What is the stock symbol for Southwest airlines?", verbose=False)


get_equity_search_symbol({'query': 'Southwest airlines', 'limit': 1}) -> [{"symbol": "LUV", "name": "SOUTHWEST AIRLINES CO", "cik": "92380"}]


The stock symbol for Southwest Airlines is **LUV**. If you have any other questions or need more details, feel free to ask!

In [35]:
agent_query(client, "What is the stock symbol for yakimee air?", verbose=False)


get_equity_search_symbol({'query': 'yakimee air', 'limit': 10}) -> []


It seems I couldn't find a stock symbol for "yakimee air." Could you please verify the company name or provide more details? Perhaps it's spelled differently or known by a different name.

In [36]:
# agent is able to first get symbol, then get annual report for symbol
agent_query(client, "What is item 1 from the 10k annual report for Microsfot?", verbose=True)



Role: You are an AI stock market assistant tasked with providing investors
with up-to-date, detailed information on individual stocks.

Objective: Assist data-driven stock market investors by giving accurate,
complete, but concise information relevant to their questions about individual
stocks.

Capabilities: You are given a number of tools as functions. Use as many tools
as needed to ensure all information provided is timely, accurate, concise,
relevant, and responsive to the user's query.

Instructions:
1. Input validation. Determine if the input is asking about a specific company
or stock ticker. If not, respond in a friendly, positive, professional tone
that you don't have information to answer and suggest alternative services
or approaches.

2. Symbol extraction. If the query is valid, extract the company name or ticker
symbol from the question. If a company name is given, look up the ticker symbol
using a tool. If the ticker symbol is not found based on the company, try to
corre

Item 1 from the latest 10-K annual report for Microsoft (symbol: MSFT) includes comprehensive details about the company, its operations, and its strategic direction. Here are key highlights:

**General Information:**
Microsoft's mission is to empower every person and organization on the planet to achieve more. The company focuses on creating platforms and tools powered by artificial intelligence (AI) to support business competitiveness, improve educational and health outcomes, and grow public-sector efficiency.

**Offerings:**
Founded in 1975, Microsoft develops and supports a range of software, services, devices, and solutions:
- Cloud-based solutions.
- Operating systems.
- Productivity and collaboration applications.
- Server applications.
- Business solutions.
- Software development tools.
- Video games.
- Devices, including PCs, tablets, gaming and entertainment consoles, and related accessories.

**Strategic Ambitions:**
Microsoft's research and development efforts are centered on:
1. Reinventing productivity and business processes.
2. Building the intelligent cloud and intelligent edge platform.
3. Creating more personal computing experiences.

**Future Opportunity:**
Microsoft is focused on the breadth and depth of the Microsoft Cloud, aiming to:
- Deliver new modern business applications.
- Enhance analytic capabilities.
- Drive innovation.
- Improve communication and collaboration.
- Simplify IT management.

**Corporate Social Responsibility:**
Microsoft commits to:
- Being a carbon negative, water positive, and zero waste company by 2030.
- Addressing racial injustice and inequity.
- Investing in digital skills development globally.

**Human Capital:**
As of June 30, 2023, Microsoft employed approximately 221,000 people worldwide. The company emphasizes a growth mindset culture, diversity, inclusion, and well-being.

This summary provides an overview of Microsoft's comprehensive operating framework and focus areas as detailed in their latest 10-K report.

In [37]:
fn_metadata = {
    "name": "get_equity_price_quote",
    "description": "Given a stock symbol, get latest market data including last price in JSON format.",
    "openapi_path" : '/api/v1/equity/price/quote',
    "parameters": {
        "symbol": {
            "type": "string",
            "description": "The stock symbol to get quote data for."
            },
        },
    "default_parameters": {
        "provider": "yfinance"
    },
    "example_parameter_values": [{
        "symbol": "AVGO",
    }],
    # "singular": 1,
}

tool = BB_agent_tool(**fn_metadata)

In [38]:
agent_query(client, f"What is the last market quote for symbol {symbol}?", verbose=False)


get_equity_price_quote({'symbol': 'MRK'}) -> [{"symbol": "MRK", "asset_type": "EQUITY", "name": "Merck & Co., Inc.", "exchange": "NYQ", "bid": 12...


The latest market quote for Merck & Co., Inc. (symbol: MRK) is as follows:
- **Last Price:** \\$129.49
- **Bid:** \\$129.35 (800 shares)
- **Ask:** \\$129.49 (1800 shares)
- **Open:** \\$131.45
- **High:** \\$131.45
- **Low:** \\$129.16
- **Previous Close:** \\$131.09
- **Volume:** 5,946,890 shares
- **52-Week High:** \\$133.10
- **52-Week Low:** \\$99.14
- **50-Day Moving Average:** \\$127.94
- **200-Day Moving Average:** \\$115.44

All prices are in USD.

In [39]:
agent_query(client, f"What is the last market quote for Consolidated Agglomerators", verbose=False)


get_equity_search_symbol({'query': 'Consolidated Agglomerators', 'limit': 1}) -> []


I couldn't locate any information for "Consolidated Agglomerators." Could you please double-check the company name or provide the stock ticker symbol if you have it? This will help me provide you with the accurate market quote.

In [40]:
agent_query(client, f"What is the current weather in Las Vegas", verbose=False)


I'm designed to assist with stock market information, and it looks like your question is about the weather. For weather updates, you might find services like weather websites or weather mobile apps helpful. Let me know if you have any stock-related questions I can help with!

In [41]:
tool = BB_agent_tool(
    name="get_company_profile_json",
    description="Given a stock symbol, get general background data about the company such as company name, industry, and sector data in JSON format",
    openapi_path='/api/v1/equity/profile',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    default_parameters={
    },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(client, f"Can you provide a basic company profile of symbol {symbol}", verbose=False)


get_company_profile_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "name": "Merck & Co., Inc.", "cik": "0000310158", "cusip": "58933Y105", "isin": "...


Here is the basic company profile for Merck & Co., Inc. (symbol: MRK):

- **Name**: Merck & Co., Inc.
- **CEO**: Mr. Robert M. Davis J.D.
- **Website**: [www.merck.com](https://www.merck.com)
- **Headquarters**: 2000 Galloping Hill Road, Kenilworth, NJ 07033, United States
- **Phone**: 908 740 4000
- **Sector**: Healthcare
- **Industry**: Drug Manufacturers - General
- **Stock Exchange**: New York Stock Exchange
- **Market Cap**: \\$327,973,566,900
- **Last Price**: \\$129.49
- **Year High**: \\$133.1
- **Year Low**: \\$99.14
- **Average Volume**: 8,199,988
- **Annualized Dividend Amount**: \\$3.08
- **Beta**: 0.402
- **Employees**: 70,000

### Description:
Merck & Co., Inc. operates as a healthcare company worldwide and has two primary segments:
1. **Pharmaceutical**: This segment offers human health pharmaceutical products in areas such as oncology, hospital acute care, immunology, neuroscience, virology, cardiovascular, and diabetes, including vaccine products for pediatrics, adolescents, and adults.
2. **Animal Health**: This segment involves the discovery, development, manufacturing, and marketing of veterinary pharmaceuticals, vaccines, and health management solutions.

The company collaborates with other major firms like AstraZeneca PLC, Bayer AG, Eisai Co., Ltd., Ridgeback Biotherapeutics, and Gilead Sciences, Inc. for developing and commercializing long-acting treatments in HIV. Founded in 1891, Merck is headquartered in Kenilworth, New Jersey.

In [42]:
# tool = BB_agent_tool(
#     name="get_equity_shorts_short_interest",
#     description="Given a stock symbol, get data on short volume and days to cover in JSON format.",
#     openapi_path='/api/v1/equity/shorts/short_interest',
#     parameters={
#         "symbol": {
#             "type": "string",
#             "description": "The stock symbol."
#         }
#         },
#     default_parameters={
#     },    
#     example_parameter_values=[{
#         "symbol": "NVDA",
#     }],
# )

# agent_query(client, f"Provide latest statistics on short interest for symbol {symbol}", verbose=False)


In [43]:
tool = BB_agent_tool(
    name="get_equity_fundamental_historical_splits",
    description="Given a stock symbol, get the company's historical stock splits in JSON format.",
    openapi_path='/api/v1/equity/fundamental/historical_splits',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    default_parameters={
    },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)


In [44]:
agent_query(client, f"Provide historical split information for symbol AAPL", verbose=False)

get_equity_fundamental_historical_splits({'symbol': 'AAPL'}) -> [{"date": "2020-08-31", "numerator": 4.0, "denominator": 1.0, "split_ratio": null, "label": "August ...


Here are the historical stock splits for Apple Inc. (AAPL):

1. **August 31, 2020**: 4-for-1
2. **June 9, 2014**: 7-for-1
3. **February 28, 2005**: 2-for-1
4. **June 21, 2000**: 2-for-1
5. **June 16, 1987**: 2-for-1

In [45]:
tool = BB_agent_tool(
    name="get_balance_sheet_json",
    description="Given a stock symbol, get the latest balance sheet data with assets and liabilities for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/balance',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
    },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    default_parameters={
    },
    singular=1
)

agent_query(client, f"what are the latest total assets for symbol {symbol}", verbose=False)


get_balance_sheet_json({'symbol': 'MRK'}) -> {"period_ending": "2023-12-31", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2024-04-...


The latest total assets for Merck & Co., Inc. (symbol: MRK) as of their fiscal year ending on December 31, 2023, is \\$106.675 billion USD.

For more details, you can refer to their latest filing [here](https://www.sec.gov/Archives/edgar/data/310158/000119312524093194/d807955d10ka.htm).

In [46]:
# can give by year
# obb.equity.fundamental.balance(symbol='NVDA', period='annual', fiscal_year='2023', limit=1)
# also growth from prior period obb.equity.fundamental.balance_growth

In [47]:
tool = BB_agent_tool(
    name="get_cash_flow_json",
    description="Given a stock symbol, get the latest cash flow statement data for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/cash',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    default_parameters={
    },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(client, f"what was the most recent cash flow from operations for stock symbol {symbol}", verbose=False)


get_cash_flow_json({'symbol': 'MRK'}) -> {"period_ending": "2023-12-31", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2024-04-...


The most recent cash flow from operations for Merck & Co., Inc. (symbol: MRK) for the fiscal year ending on December 31, 2023, was \\$13,006,000,000.

For more detailed information, you can view their [report on the SEC website](https://www.sec.gov/Archives/edgar/data/310158/000119312524093194/d807955d10ka.htm).

In [48]:
tool = BB_agent_tool(
    name="get_income_statement_json",
    description="Given a stock symbol, get the latest income statement data for the company in JSON format",
    openapi_path='/api/v1/equity/fundamental/income',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    default_parameters={
    },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(client, f"what was the most recent net income for symbol {symbol}", verbose=False)


get_equity_search_symbol({'query': 'MRK', 'limit': 10}) -> [{"symbol": "MRK", "name": "Merck & Co., Inc.", "cik": "310158"}, {"symbol": "NMRK", "name": "NEWMAR...
get_income_statement_json({'symbol': 'MRK'}) -> {"period_ending": "2023-12-31", "fiscal_period": "FY", "fiscal_year": 2023, "filing_date": "2024-04-...


The most recent net income for Merck & Co., Inc (symbol: MRK) was \\$365 million for the fiscal year ending December 31, 2023. 

Here is the link to the detailed income statement:
[SEC Filing](https://www.sec.gov/Archives/edgar/data/310158/000119312524093194/0001193125-24-093194-index.htm)

In [49]:
tool = BB_agent_tool(
    name="get_fundamental_metrics_json",
    description="Given a stock symbol, get fundamental metrics for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/metrics',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    default_parameters={
    },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(client, f"what was the most recent PE ratio for stock symbol {symbol}", verbose=False)


get_fundamental_metrics_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "market_cap": 276583740000.0, "pe_ratio": 757.7636712328766, "period_ending": "20...


The most recent PE ratio for Merck & Co., Inc. (symbol: MRK) as of December 31, 2023, was approximately **757.76**.

In [50]:
tool = BB_agent_tool(
    name="get_fundamental_ratios_json",
    description="Given a stock symbol, get fundamental valuation ratios for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/ratios',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    default_parameters={
    },
    singular=1
)

agent_query(client, f"what was the most recent price to sales ratio for stock symbol {symbol}", verbose=False)


get_fundamental_metrics_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "market_cap": 276583740000.0, "pe_ratio": 757.7636712328766, "period_ending": "20...


The most recent price-to-sales (P/S) ratio for Merck & Co, Inc. (symbol: MRK) for the fiscal year ending on December 31, 2023, is 4.62.

In [51]:
tool = BB_agent_tool(
    name="get_equity_fundamental_multiples",
    description="Given a stock symbol, get fundamental valuation multiples for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/multiples',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    default_parameters={
    },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
    singular=1
)

agent_query(client, f"what was the most recent revenue per share for stock symbol {symbol}", verbose=False)


get_fundamental_metrics_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "market_cap": 276583740000.0, "pe_ratio": 757.7636712328766, "period_ending": "20...


The most recent revenue per share for Merck & Co., Inc. (symbol: MRK) for the fiscal year ending December 31, 2023, was \\$23.60.

In [52]:
# tool = BB_agent_tool(
#     name="get_historical_eps",
#     description="Given a stock symbol, get historical earnings per share data for the company in JSON format.",
#     openapi_path='/api/v1/equity/fundamental/historical_eps',
#     parameters={
#         "symbol": {
#             "type": "string",
#             "description": "The stock symbol."
#         }
#         },
#     example_parameter_values=[{
#         "symbol": "NVDA",
#     }],
# )

#

In [53]:
# agent_query(client, f"what was the EPS for {company} in the quarter ended 2022-09-30?", verbose=False)


In [54]:
tool = BB_agent_tool(
    name="get_equity_fundamental_dividend",
    description="Given a stock symbol, get the latest dividend data for the company in JSON format.",
    openapi_path='/api/v1/equity/fundamental/dividends',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    default_parameters={
    },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(client, f"what was the latest dividend data for stock symbol {symbol}?", verbose=False)


get_equity_search_symbol({'query': 'MRK', 'limit': 1}) -> [{"symbol": "MRK", "name": "Merck & Co., Inc.", "cik": "310158"}, {"symbol": "NMRK", "name": "NEWMAR...
get_equity_fundamental_dividend({'symbol': 'MRK'}) -> [{"ex_dividend_date": "2024-03-14", "amount": 0.77, "label": "March 14, 24", "adj_dividend": 0.77, "...


The latest dividend data for Merck & Co., Inc. (symbol: MRK) is as follows:

- **Ex-Dividend Date:** March 14, 2024
- **Amount:** \\$0.77
- **Record Date:** March 15, 2024
- **Payment Date:** April 5, 2024
- **Declaration Date:** January 23, 2024

In [55]:
agent_query(client, f"what was the dividend data as of the end of 2023 for stock symbol {symbol}?", verbose=False)


get_equity_search_symbol({'query': 'MRK', 'limit': 1}) -> [{"symbol": "MRK", "name": "Merck & Co., Inc.", "cik": "310158"}, {"symbol": "NMRK", "name": "NEWMAR...
get_equity_fundamental_dividend({'symbol': 'MRK'}) -> [{"ex_dividend_date": "2024-03-14", "amount": 0.77, "label": "March 14, 24", "adj_dividend": 0.77, "...


As of the end of 2023, here is the dividend data for Merck & Co., Inc. (symbol: MRK):

- **Ex-dividend date:** December 14, 2023
- **Amount:** \\$0.77 per share
- **Record date:** December 15, 2023
- **Payment date:** January 8, 2024
- **Declaration date:** November 28, 2023

If you need any more information, feel free to ask!

In [56]:
tool = BB_agent_tool(
    name="get_trailing_dividend_yield_json",
    description="Given a stock symbol, get the 1 year trailing dividend yield for the company over time in JSON format.",
    openapi_path='/api/v1/equity/fundamental/trailing_dividend_yield',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    default_parameters={
    },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(client, f"what was the dividend yield for stock symbol {symbol} as of the end of 2023?", verbose=False)


get_trailing_dividend_yield_json({'symbol': 'MRK'}) -> [{"date": "2023-05-25", "trailing_dividend_yield": 0.0252894034}, {"date": "2023-05-26", "trailing_d...


The trailing dividend yield for Merck & Co., Inc. (symbol: MRK) as of the end of 2023 was approximately 2.79%.

In [57]:
tool = BB_agent_tool(
    name="get_price_performance_json",
    description="Given a stock symbol, get price performance data for the stock for different time periods in JSON format.",
    openapi_path='/api/v1/equity/price/performance',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    default_parameters={
    },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)

agent_query(client, f"what was the performance for {company} from 1 year ago?", verbose=False)


get_equity_search_symbol({'query': 'Merck', 'limit': 1}) -> [{"symbol": "MRK", "name": "Merck & Co., Inc.", "cik": "310158"}]
get_price_performance_json({'symbol': 'MRK'}) -> [{"symbol": "MRK", "one_day": -0.012204999999999999, "wtd": null, "one_week": -0.010091000000000001,...


The 1-year stock performance for Merck & Co., Inc. (symbol: MRK) is +16.58%.

If you need any further details, feel free to ask!

In [58]:
# this is unreliable, chatgpt can't always find the higest value in a large dict
tool = BB_agent_tool(
    name="get_etf_equity_exposure_json",
    description="Given a stock symbol, get the exposure of ETFs to the stock in JSON format.",
    openapi_path='/api/v1/etf/equity_exposure',
    parameters={
        "symbol": {
            "type": "string",
            "description": "The stock symbol."
        }
        },
    default_parameters={
    },
    example_parameter_values=[{
        "symbol": "NVDA",
    }],
)


In [59]:
pd.DataFrame(json.loads(tool(symbol='MRK'))).sort_values('weight', ascending=False)

,equity_symbol,etf_symbol,shares,weight,market_value
437,MRK,FTXH,11684.0,0.075700,1.512961e+06
84,MRK,IUHC.L,1132285.0,0.061898,1.484312e+08
13,MRK,XLV,18406426.0,0.061826,2.383448e+09
81,MRK,QDVG.DE,1134961.0,0.061364,1.488955e+08
83,MRK,IUHE.AS,0.0,0.061238,1.484426e+08
...,...,...,...,...,...
665,MRK,GPAL,640.0,NaN,6.631040e+04
677,MRK,GDEF,397.0,NaN,5.204273e+04
685,MRK,EUNU.DE,50000.0,NaN,3.868500e+04
698,MRK,GSEU,121.0,NaN,2.214566e+04


In [60]:
agent_query(client, f"which ETF has the highest weight in {company} ?", verbose=False)


get_equity_search_symbol({'query': 'Merck', 'limit': 1}) -> [{"symbol": "MRK", "name": "Merck & Co., Inc.", "cik": "310158"}]
get_etf_equity_exposure_json({'symbol': 'MRK'}) -> [{"equity_symbol": "MRK", "etf_symbol": "VTSAX", "shares": 79744771.0, "weight": 0.0067, "market_val...


The ETF with the highest weight in Merck (symbol: MRK) is:

- **iShares U.S. Pharmaceuticals ETF (IYH)** with a weight of **5.99559%**

### ETF Details
- **ETF Symbol:** IYH
- **Shares:** 1,526,770
- **Market Value:** \\\$200,144,279.30

This makes IYH the ETF with the highest exposure to Merck.

In [61]:
# make a custom tool
# really might as well write a function to return top 10 etfs with highest weight in company

# def get_largest_from_json(json_str, field_name):
#     d = json.loads(json_str)
#     df = pd.DataFrame(d).sort_values(field_name, ascending=False)
#     return [{'largest': df.iloc[0].to_json()}]


# tool = BB_agent_tool(
#     name="get_largest_from_json",
#     description="Given a json string json_str representing a list of objects, and a field name field_name, return the object with the largest value for field_name.",
#     openapi_path=None,
#     callable=get_largest_from_json,
#     parameters={
#         "json_str": {
#             "type": "string",
#             "description": "a JSON string representing a list of objects"
#             },
#         "field_name": {
#             "type": "string",
#             "description": "the field to search for the largest value"
#             },
#         },
#     default_parameters={
#     },
#     example_parameter_values=[{
#         "json_str": '[{"name": "low", "value": 0}, {"name": "high", "value": 1}]',
#         "field_name": 'value'
#     }],
# )

# tool_response = tool(json_str='[{"name": "low", "value": 0}, {"name": "high", "value": 1}]',
#                     field_name='value')
# tool_response
# len(json.loads(tool_response)[0]['item1'])


In [62]:
[(k, t.description) for k, t in BB_agent_tool.agent_registry.items()]

[('get_10k_item1_from_symbol',
  "Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing."),
 ('get_equity_search_symbol',
  'Given a search string, get the stock symbol of the top company whose name best matches the search string.'),
 ('get_equity_price_quote',
  'Given a stock symbol, get latest market data including last price in JSON format.'),
 ('get_company_profile_json',
  'Given a stock symbol, get general background data about the company such as company name, industry, and sector data in JSON format'),
 ('get_equity_fundamental_historical_splits',
  "Given a stock symbol, get the company's historical stock splits in JSON format."),
 ('get_balance_sheet_json',
  'Given a stock symbol, get the latest balance sheet data with assets and liabilities for the company in JSON format.'),
 ('get_cash_flow_json',
  'Given a stock symbol, get the latest cash flow statement data for the company in JSON format.'),
 ('get_income_statement_json',
  'Given a stock

In [63]:
# for k, t in BB_agent_tool.agent_registry.items():
#     print(t.example_code)

In [64]:
len(" ".join([(t.example_code) for k, t in BB_agent_tool.agent_registry.items()]))

23792